<a href="https://colab.research.google.com/github/jmsarmiento11/csc612m-dot-product/blob/master/CUDA_Dot_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###DOTP C++ program version

In [13]:
%%writefile c_dotProduct.c
//Mark Jimbo Sarmiento, MSCS
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

extern void SIMDdotProduct(size_t ARRAY_SIZE, float* A, float* B,float* sdot);
extern void x86dotProduct(size_t ARRAY_SIZE, float* A, float* B, float* sdot);

const int ARRAY_SIZE = 2<<28;

void dotProduct(const float* A, const float* B, float* sdot)
{
    for (int i = 0; i < ARRAY_SIZE; i++) {
        sdot[i] = A[i] * B[i];
    }

    // add up the elements of the sdot array
    float sum = 0;
    for (int i = 0; i < ARRAY_SIZE; i++) {
        sum += sdot[i];
    }

    //printf("Dot product result: %f\n", sum);
    //return 0;
}

void verifyDotProduct(const float* sdot, const float* expected)
{
    for (int i = 0; i < ARRAY_SIZE; i++) {
        if (fabs(sdot[i] - expected[i]) > 1e-5) {
            printf("Error found at index %d\n", i);
            return;
        }
    }
    printf("No error.\n");
}

int main()
{
    float* A = (float*)malloc(ARRAY_SIZE * sizeof(float));
    float* B = (float*)malloc(ARRAY_SIZE * sizeof(float));
    float* sdot = (float*)malloc(ARRAY_SIZE * sizeof(float));
    float* expected = (float*)malloc(ARRAY_SIZE * sizeof(float));

    // --------------------- C++ version ----------------------------------


    //flush out cache
    dotProduct(A, B, sdot);

    // fill in the host memory with data
    for (int i = 0; i < ARRAY_SIZE; i++) {
        A[i] = i;
        B[i] = i;
        expected[i] = A[i] * B[i]; // compute the expected result on the CPU
    }

    // Measure execution time
    clock_t start = clock();

    dotProduct(A, B, sdot);

    // verify the correctness of the dot product
    verifyDotProduct(sdot, expected);

    // finish up on the CPU side
    float sum = 0;
    for (int i = 0; i < ARRAY_SIZE; i++) {
        sum += sdot[i];
    }

    clock_t end = clock();
    double time_taken = ((double)(end - start)) * 1e6 / CLOCKS_PER_SEC;

    printf("Dot product result: %f\n", sum);
    printf("C function took %f microseconds for array size %d \n", time_taken, ARRAY_SIZE);
// free memory
    free(A);
    free(B);
    free(sdot);
    free(expected);

    return 0;
}

Overwriting c_dotProduct.c


In [14]:
%%shell
g++ c_dotProduct.c -o c_dotProduct

In [15]:
%%shell
./c_dotProduct

No error.
Dot product result: 4835703278458516698824704.000000
C function took 7239440.000000 microseconds for array size 536870912 


###DOTP CUDA version 1.0 : grid-stride loop with prefetching + mem advise

In [16]:
%%writefile dotProdGridStride.cu
// Mark Jimbo Sarmiento, MSCS
#include <stdio.h>
#include <stdlib.h>

const int ARRAY_SIZE = 2 << 28;
//const int numThreadsPerBlock = 256;

__global__ void dotProductKernel(int size, float* sdot, const float* A, const float* B) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    int stride = blockDim.x * gridDim.x;

    // each thread computes a dot product of two elements
    while (index < size) {
        sdot[index] = A[index] * B[index];
        index += stride;
    }
}

void verifyDotProduct(const float* sdot, const float* expected) {
    for (int i = 0; i < ARRAY_SIZE; i++) {
        if (fabs(sdot[i] - expected[i]) > 1e-5) {
            printf("Error found at index %d\n", i);
            return;
        }
    }
    printf("No error.\n");
}

int main() {
    const unsigned ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

    // Declare array
    float* A, * B, * sdot;
    cudaMallocManaged(&A, ARRAY_BYTES);
    cudaMallocManaged(&B, ARRAY_BYTES);
    cudaMallocManaged(&sdot, ARRAY_BYTES);

    // Fill the host memory with data
    for (int i = 0; i < ARRAY_SIZE; i++) {
        A[i] = float(i);
        B[i] = float(i);
    }

    // Call CUDA kernel
    int numBlocks = 1024;
    int numThreads = 1024;
    printf("numBlocks = %d, numThreads = %d\n", numBlocks, numThreads);
    dotProductKernel<<<numBlocks, numThreads>>>(ARRAY_SIZE, sdot, A, B);

    // Barrier
    cudaDeviceSynchronize();

    // Display the dot product result
    float sum = 0.0f;
    for (int i = 0; i < ARRAY_SIZE; i++) {
        sum += sdot[i];
    }
    printf("Dot product result: %f\n", sum);

    // Verify the correctness of the dot product
    float* expected = (float*)malloc(ARRAY_SIZE * sizeof(float));
    for (int i = 0; i < ARRAY_SIZE; i++) {
        expected[i] = A[i] * B[i]; // compute the expected result on the CPU
    }
    verifyDotProduct(sdot, expected);
    free(expected);

    // Free memory
    cudaFree(A);
    cudaFree(B);
    cudaFree(sdot);

    return 0;
}


Overwriting dotProdGridStride.cu


In [17]:
%%shell
nvcc dotProdGridStride.cu -o dotProdGridStride

In [18]:
%%shell
nvprof ./dotProdGridStride

==5306== NVPROF is profiling process 5306, command: ./dotProdGridStride
numBlocks = 1024, numThreads = 1024
Dot product result: 4835703278458516698824704.000000
No error.
==5306== Profiling application: ./dotProdGridStride
==5306== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.03387s         1  2.03387s  2.03387s  2.03387s  dotProductKernel(int, float*, float const *, float const *)
      API calls:   75.86%  2.03387s         1  2.03387s  2.03387s  2.03387s  cudaDeviceSynchronize
                   18.71%  501.53ms         3  167.18ms  162.45ms  172.10ms  cudaFree
                    5.42%  145.31ms         3  48.437ms  17.899us  145.23ms  cudaMallocManaged
                    0.00%  113.55us       101  1.1240us     137ns  46.667us  cuDeviceGetAttribute
                    0.00%  78.494us         1  78.494us  78.494us  78.494us  cudaLaunchKernel
                    0.00%  29.195us         1  29.195us  29